## Rubric

Instructions: DELETE this cell before you submit via a `git push` to your repo before deadline. This cell is for your reference only and is not needed in your report. 

Scoring: Out of 10 points

- Each Developing  => -2 pts
- Each Unsatisfactory/Missing => -4 pts
  - until the score is 

If students address the detailed feedback in a future checkpoint they will earn these points back


|                  | Unsatisfactory                                                                                                                                                                                                    | Developing                                                                                                                                                                                              | Proficient                                     | Excellent                                                                                                                              |
|------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------|
| Data relevance   | Did not have data relevant to their question. Or the datasets don't work together because there is no way to line them up against each other. If there are multiple datasets, most of them have this trouble | Data was only tangentially relevant to the question or a bad proxy for the question. If there are multiple datasets, some of them may be irrelevant or can't be easily combined.                       | All data sources are relevant to the question. | Multiple data sources for each aspect of the project. It's clear how the data supports the needs of the project.                         |
| Data description | Dataset or its cleaning procedures are not described. If there are multiple datasets, most have this trouble                                                                                              | Data was not fully described. If there are multiple datasets, some of them are not fully described                                                                                                      | Data was fully described                       | The details of the data descriptions and perhaps some very basic EDA also make it clear how the data supports the needs of the project. |
| Data wrangling   | Did not obtain data. They did not clean/tidy the data they obtained.  If there are multiple datasets, most have this trouble                                                                                 | Data was partially cleaned or tidied. Perhaps you struggled to verify that the data was clean because they did not present it well. If there are multiple datasets, some have this trouble | The data is cleaned and tidied.                | The data is spotless and they used tools to visualize the data cleanliness and you were convinced at first glance                      |


# COGS 108 - Data Checkpoint

## Authors

Instructions: REPLACE the contents of this cell with your team list and their contributions. Note that this will change over the course of the checkpoints

This is a modified [CRediT taxonomy of contributions](https://credit.niso.org). For each group member please list how they contributed to this project using these terms:
> Analysis, Background research, Conceptualization, Data curation, Experimental investigation, Methodology, Project administration, Software, Visualization, Writing – original draft, Writing – review & editing

Example team list and credits:
- Alice Anderson: Conceptualization, Data curation, Methodology, Writing - original draft
- Bob Barker:  Analysis, Software, Visualization
- Charlie Chang: Project administration, Software, Writing - review & editing
- Dani Delgado: Analysis, Background research, Visualization, Writing - original draft

## Research Question

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback



## Background and Prior Work

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback

## Hypothesis


Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback


## Data

### Data overview

Instructions: REPLACE the contents of this cell with descriptions of your actual datasets.

For each dataset include the following information
- Dataset #1
  - Dataset Name:
  - Link to the dataset:
  - Number of observations:
  - Number of variables:
  - Description of the variables most relevant to this project
  - Descriptions of any shortcomings this dataset has with repsect to the project
- Dataset #2 (if you have more than one!)
  - same as above
- etc

Each dataset deserves either a set of bullet points as above or a few sentences if you prefer that method.

If you plan to use multiple datasets, add a few sentences about how you plan to combine these datasets.

In [5]:
# Run this code every time when you're actively developing modules in .py files.  It's not needed if you aren't making modules
#
## this code is necessary for making sure that any modules we load are updated here 
## when their source code .py files are modified

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# Setup code -- this only needs to be run once after cloning the repo!
# this code downloads the data from its source to the `data/00-raw/` directory
# if the data hasn't updated you don't need to do this again!

# if you don't already have these packages (you should!) uncomment this line
# %pip install requests tqdm

import sys
sys.path.append('./modules') # this tells python where to look for modules to import

import get_data # this is where we get the function we need to download data

# replace the urls and filenames in this list with your actual datafiles
# yes you can use Google drive share links or whatever
# format is a list of dictionaries; 
# each dict has keys of 
#   'url' where the resource is located
#   'filename' for the local filename where it will be stored 
datafiles = [
    {
        "url": "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2024.csv",
        "filename": "atp_matches_2024.csv",
    }
]

get_data.get_raw(datafiles, destination_directory="data/00-raw/")


Overall Download Progress: 100%|██████████| 1/1 [00:00<00:00,  3.00it/s]    

Successfully downloaded: atp_matches_2024.csv


### ATP Men’s Tennis Matches – 2024 Season 

The ATP Matches 2024 dataset contains detailed match-level information for all ATP men’s professional tennis matches played between January 1 and December 31, 2024. The data comes from Jeff Sackmann’s tennis_atp repository and includes 3,076 matches and 49 variables. Each row represents a single tennis match and each column represents one variable, meaning the dataset is already in tidy format.

The dataset includes several important categories of variables. Tournament information includes surface (Hard, Clay, Grass), tournament level (Grand Slam, Masters 1000, ATP 250/500), and date. Player information includes winner and loser names, handedness, height (cm), and age (years). Match outcome variables include score, round, best_of (3 or 5 sets), and match duration in minutes.

Several performance metrics are particularly important:

Aces (w_ace, l_ace): Number of serves that result in an immediate point. Measured as a count per match. Professional players typically average 5–15 aces per match. Higher ace counts generally indicate stronger serving performance.

Double Faults (w_df, l_df): Number of missed second serves. Measured as a count per match. Professionals usually aim to stay under 3–5 per match since double faults directly give points to the opponent.

Match Duration (minutes): Measured in minutes. Best-of-three matches typically last 90–150 minutes; best-of-five matches may exceed 3–5 hours.

ATP Ranking (winner_rank, loser_rank): Numerical ranking (1 = best). Rankings are based on a rolling 52-week point system. Lower numbers indicate stronger players.

Dataset Concerns:

Approximately 7% of all values are missing. Missingness is not random. It is strongly associated with incomplete matches (retirements and walkovers). There are 101 matches marked as walkover (W/O) or retirement (RET), and 238 matches missing duration data. Around 60 matches are missing serve statistics such as aces and double faults. These missing values are systematic and expected because incomplete matches do not generate full statistics.

There are also extreme match duration values (e.g., 0 minutes for walkovers and matches exceeding 220 minutes), but these represent legitimate real-world scenarios and should not automatically be removed.

In [7]:
import pandas as pd
import numpy as np
from pathlib import Path

# ----------------------------
# Locate and Load Raw Data
# ----------------------------
raw_path = Path("data/00-raw/atp_matches_2024.csv")

if not raw_path.exists():
    hits = list(Path(".").rglob("atp_matches_2024.csv"))
    if not hits:
        raise FileNotFoundError(
            "Could not find atp_matches_2024.csv. "
            "Make sure you ran the download cell or placed the file in data/00-raw/."
        )
    raw_path = hits[0]

df = pd.read_csv(raw_path)

print("Loaded from:", raw_path)
print("Dataset shape (rows, columns):", df.shape)


# ----------------------------
# Check Tidiness
# ----------------------------
print("\nDuplicate rows:", df.duplicated().sum())


# ----------------------------
# Missing Data Analysis
# ----------------------------
missing_counts = df.isna().sum()
missing_percent = (missing_counts / len(df)) * 100

missing_summary = pd.DataFrame({
    "Missing_Count": missing_counts,
    "Missing_Percent": missing_percent
}).sort_values(by="Missing_Percent", ascending=False)

print("\nTop Missing Columns:")
print(missing_summary.head(15))

print("\nTotal Missing Values:", df.isna().sum().sum())


# ----------------------------
# Demonstrate Systematic Missingness
# ----------------------------
df["score_ret_wo"] = df["score"].str.contains("RET|W/O", na=False)

print("\nCross-tab: Missing Minutes vs RET/W/O")
print(pd.crosstab(df["minutes"].isna(), df["score_ret_wo"], margins=True))


# ----------------------------
# Flag Incomplete Matches
# ----------------------------
stat_cols = ["minutes", "w_ace", "l_ace", "w_df", "l_df"]

df["incomplete_match"] = (
    df["score"].str.contains("RET|W/O", na=False) |
    df[stat_cols].isna().any(axis=1)
)

print("\nNumber of Incomplete Matches:", df["incomplete_match"].sum())


# ----------------------------
# Outlier Detection (Match Duration IQR)
# ----------------------------
minutes_clean = df["minutes"].dropna()

Q1 = minutes_clean.quantile(0.25)
Q3 = minutes_clean.quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[
    (df["minutes"] < lower_bound) |
    (df["minutes"] > upper_bound)
]

print("\nMatch Duration IQR Bounds:", lower_bound, upper_bound)
print("Number of Duration Outliers:", len(outliers))


# ----------------------------
# Save Processed Dataset
# ----------------------------
processed_path = Path("data/02-processed/atp_matches_2024_processed.csv")
processed_path.parent.mkdir(parents=True, exist_ok=True)

df.to_csv(processed_path, index=False)

print("\nProcessed dataset saved to:", processed_path)


# ----------------------------
# Summary Statistics
# ----------------------------
print("\nMatch Duration Summary:")
print(df["minutes"].describe())

print("\nWinner Aces Summary:")
print(df["w_ace"].describe())

print("\nWinner Rank Summary:")
print(df["winner_rank"].describe())

Loaded from: data/00-raw/atp_matches_2024.csv
Dataset shape (rows, columns): (3076, 49)

Duplicate rows: 0

Top Missing Columns:
              Missing_Count  Missing_Percent
winner_entry           2599        84.492848
loser_entry            2358        76.657997
loser_seed             2319        75.390117
winner_seed            1782        57.932380
minutes                 238         7.737321
l_SvGms                  61         1.983095
w_SvGms                  61         1.983095
l_svpt                   60         1.950585
w_svpt                   60         1.950585
w_df                     60         1.950585
w_ace                    60         1.950585
w_2ndWon                 60         1.950585
w_bpSaved                60         1.950585
w_bpFaced                60         1.950585
l_ace                    60         1.950585

Total Missing Values: 10563

Cross-tab: Missing Minutes vs RET/W/O
score_ret_wo  False  True   All
minutes                        
False          2750

### Dataset #2 

See instructions above for Dataset #1.  Feel free to keep adding as many more datasets as you need.  Put each new dataset in its own section just like these. 

Lastly if you do have multiple datasets, add another section where you demonstrate how you will join, align, cross-reference or whatever to combine data from the different datasets

Please note that you can always keep adding more datasets in the future if these datasets you turn in for the checkpoint aren't sufficient.  The goal here is demonstrate that you can obtain and wrangle data.  You are not tied down to only use what you turn in right now.

In [ ]:
## YOUR CODE TO LOAD/CLEAN/TIDY/WRANGLE THE DATA GOES HERE


## Ethics

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback

## Team Expectations 

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback


## Project Timeline Proposal

Instructions: Replace this with your timeline.  **PLEASE UPDATE your Timeline!** No battle plan survives contact with the enemy, so make sure we understand how your plans have changed.  Also if you have lost points on the previous checkpoint fix them